# Analiza podatkov - Cene stanovanj v Sloveniji



## Projektna naloga pri predmetu Programiranje 1

Naloga obsega analizo podatkov o stanovanjih, ki so naprodaj v Sloveniji, pridobljenih iz spletne strani www.nepremicnine.net/ v mesecu novembru 2021 ter januarju 2022.

Za posamezne regije oz. območja, kot so navedena na spletni strani sem zajela naslednje podatke: 
- lokacija
- število sob
- nadstropje
- leto izgradnje
- velikost
- cena

Žal sem podatke zajela le v dveh mesecih, zanimiva bi bila podobna analiza za vse mesece v letu, morda tudi primerjava z drugimi državami. V tej nalogi bom postavila ogrodje za takšno analizo na pridobljenih podatkih.

In [158]:
import pandas as pd

In [159]:
path_november = "C:/Users/Milka/Cene-stanovanj-v-Sloveniji/csv_podatki_november/"

path_januar = "C:/Users/Milka/Cene-stanovanj-v-Sloveniji/csv_podatki_januar/"

import os 
seznam_november = os.listdir(path_november)
seznam_januar = os.listdir(path_januar)

def tabele_v_seznam(seznam, path):
    seznam_tabel = []
    for i in range(len(seznam)):
        tabela = pd.read_csv(path + seznam[i])
        seznam_tabel.append(tabela)
    return seznam_tabel

Za lažje delo bom najprej vse podatke za posamezni mesec združila v novo tabelo. Vsakemu stanovanju bo dodan nov stolpec z informacijo o regiji.

In [160]:
tabele_november = tabele_v_seznam(seznam_november, path_november)
tabele_januar = tabele_v_seznam(seznam_januar, path_januar)

stolpec_regij = ["Dolenjska", "Gorenjska", "Južna Primorska", 
"Koroška", "LJ center", "LJ okolica", "Posavska", 
"Savinjska", "Severna Primorska", "Zasavska"]

podatki_november = pd.concat(tabele_november, keys=stolpec_regij)
podatki_januar = pd.concat(tabele_januar, keys=stolpec_regij)

### Čiščenje podatkov
V eni izemd tabel se je pojavil dodaten stolpec opis, ki ni koristen, zato ga izbrišem. 
Opazila sem, da se v csv datoteki ponavljajo vnosi, čeprav na spletni strani ni ponovljenih oglasov.
Pri združevanju tabel in brisanju vrstic so se pojavile težave pri indeksih.

In [161]:
pd.set_option("display.max_rows", None, "display.max_columns", 20)

november = podatki_november.drop_duplicates().drop(columns= 'opis')
januar = podatki_januar.drop_duplicates()

november = november.reset_index().drop('level_1', 1)
januar = januar.reset_index().drop('level_1', 1)

november['regija'] = november['level_0']
januar['regija'] = januar['level_0']

november = november.drop('level_0', 1)
januar = januar.drop('level_0', 1)

#V tabeli januar se je na enem mestu 64 pojavila napačna vrednost, zato jo ročno popravim

januar['cena'][364] = '199.000,00'

C:\Users\Milka\AppData\Local\Temp\ipykernel_15308\3685942428.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  november = november.reset_index().drop('level_1', 1)
C:\Users\Milka\AppData\Local\Temp\ipykernel_15308\3685942428.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  januar = januar.reset_index().drop('level_1', 1)
C:\Users\Milka\AppData\Local\Temp\ipykernel_15308\3685942428.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  november = november.drop('level_0', 1)
C:\Users\Milka\AppData\Local\Temp\ipykernel_15308\3685942428.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  januar = januar.drop('level_0', 1)
C:\Users\Milka\AppData

Pri pridobivanju podatkov se je pojavilo nekaj napak pri stolpcu 'cena'. Podatke prečistim z naslednjo funckijo in jim spremeni tip v cela števila.
V float bo potrebno spremeniti tudi velikost stanovanja, v int pa leto gradnje.

In [166]:
def popravi_ceno(vnos):
    nov_niz = ''
    for znak in str(vnos):
        if znak != ',':
            nov_niz += znak
        else:
            break
    return int(nov_niz.replace('.',''))

def popravi_velikost(vnos):
    return float(vnos.replace(',', '.'))

def popravi_leto(vnos):
    return int(vnos)

november['cena'] = november['cena'].map(lambda x: popravi_ceno(x))
januar['cena'] = januar['cena'].map(lambda x: popravi_ceno(x))

november['velikost'] = november['velikost'].map(lambda x: popravi_velikost(x))
januar['velikost'] = januar['velikost'].map(lambda x: popravi_velikost(x))

november['leto'] = november['leto'].map(lambda x: popravi_leto(x))
januar['leto'] = januar['leto'].map(lambda x: popravi_leto(x))




Za primerjavo cen bo koristna informacija o ceni na kvadratni meter stanovanja.

In [172]:
november['cena na m2'] = round(november['cena'] / november['velikost'])
januar['cena na m2'] = round(januar['cena'] / januar['velikost'])

In [173]:
# filter po starosti, velikosti
starost_januar = januar.sort_values(by=['leto'], ascending=True)
cena_januar = januar.sort_values(by=['velikost'], ascending=True)

Za natančnejšo analizo lokacij, moram počistiti stolpec, tako da bo v njem ostalo samo eno ime. 

In [176]:
def popravi_mesto(vnos):
    nov_niz = ''
    for znak in str(vnos):
        if znak != ',':
            nov_niz += znak
        else:
            break
    return nov_niz

november['mesto'] = november['lokacija'].map(lambda x: popravi_mesto(x))
januar['mesto'] = januar['lokacija'].map(lambda x: popravi_mesto(x))

# Analiza podatkov
Zdaj so podatki v grobem pripravljeni za analizo.
Najprej si poglejmo, koliko stanovanj je bilo na voljo po regijah in natančneje po mestih.

Koliko stanovanj je na voljo v posameznih regijah in razlika med meseci. (tabela)

Za vsako izmed regij izračunam povprečno ceno, povprečno ceno na kvadrat, velikost stanovanj, starost ter število le-teh. Primerjava med novembrom in januarjem za vse regije. (tri ločene tabele). Ali drži, da so stanovanja najdražja v Ljubljani? Spreminjanje cen po mesecih.

Za mesec januar ter november pogledam povprečno ceno stanovanj po posameznih mestih. Kje je stanovanj največ?

Združim tabele za november in januar po regijah (tiste, ki se ponavljajo upoštevam samo enkrat). Raziščem povezavo med starostjo in cenami stanovanj, ter med velikostjo ter cenami stanovanj. V katerih regijah je na trgu največ novejših stanovanj in kje največ starejših? Tabela št. stanovanj po letih, ter po velikosti.

Cena v odvisnosti od velikosti sob in nadstropja.

Za podatke iz novembra in januarja poiščem ponavljajoče se oglase in jih shranim v novo tabelo. Ali so stanovanja, ki so dlje časa na trgu dražja?

Zaključek: